In [ ]:
import argparse
from pathlib import Path
from win32com import client
import os
from PyPDF2 import PdfFileReader, PdfFileWriter
import json
import pandas as pd

In [ ]:
info_df = pd.read_excel('Evaluation Sheet 21-22.xlsx', sheet_name='Evaluation')[['Name', 'Campus', 'School']].dropna(subset=['Name'])

In [ ]:
info_js = {row['Name'].strip(): {"Campus": row['Campus'], "School": row['School']} for idx, row in info_df.iterrows()}

In [ ]:
def create_pdf(file_name, sheet_name_or_number):
    excel = client.Dispatch("Excel.Application")
    excel.ScreenUpdating = False
    excel.DisplayAlerts = False
    excel.EnableEvents = False
    # Read Excel File
    sheets = excel.Workbooks.Open(os.path.join(os.getcwd(), file_name + '.xlsx'))
    work_sheets = sheets.Worksheets[sheet_name_or_number]
    # Convert into PDF File
    work_sheets.ExportAsFixedFormat(0, os.path.join(os.getcwd(), file_name + ".pdf"))

In [ ]:
def split_pdf(file_name, info_json, save_file_name = None, folder_name = "Names"):
    if save_file_name == None:
        save_file_name = file_name
    output_folder_path = '{}/'.format(folder_name)
    Path(output_folder_path).mkdir(parents=True, exist_ok=True)
    print("The '{}/' folder has been created".format(folder_name))

    pdf = PdfFileReader(os.path.join(os.getcwd(), file_name + ".pdf"))

    location = None
    for page_num in range(pdf.numPages):
        pdfWriter = PdfFileWriter()
        pdfWriter.addPage(pdf.getPage(page_num))
        page_content = pdf.getPage(page_num).extractText()
        list_of_names = page_content.encode('utf-8').decode('utf-8').split("\n")
        if location == None:
            for index, name in enumerate(list_of_names):
                if 'name' in name.lower():
                    location = index + 1
                    break
        try:
            Name = list_of_names[location].strip()
        except:
            continue
            
        if 'name' not in list_of_names[location - 1].lower().strip():
            continue
            
        if Name == '#N/A':
            continue

        try:
            file_location = "{}{}/{}/{}/".format(output_folder_path, info_js[Name]['Campus'], info_js[Name]['School'], Name)
        except:
            print("\t\tCouldn't find {}".format(Name))
            continue
        print("\t" + file_location + '{}.pdf'.format(save_file_name))
        Path(file_location).mkdir(parents=True, exist_ok=True)
        with open(file_location + '{}.pdf'.format(save_file_name), 'wb') as f:
            pdfWriter.write(f)

In [ ]:
file_name = ['Evaluation Sheet 21-22.xlsx']
sheet_name = ['Evaluation Forms Fall']
for index, name in enumerate(file_name):
    create_pdf(str(Path(name).with_suffix('')), sheet_name[index])
    split_pdf(str(Path(name).with_suffix('')), info_js, 'Performance Evaluation Fall 21-22')